# Índices y Tipología de la Complejidad Electoral 2020: Municipios
### INFOTEC-Maestría en Ciencia de Datos e Información
#### Autor: Miguel David Alvarez Hernández (mdalvarezh@gmail.com)

<br>

## <center>Procesamiento de datos y construcción de indicadores<center>

In [1]:
import datetime
now = datetime.datetime.now()
print ("Última versión:")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Última versión:
2020-06-26 12:04:14


## Setup

In [2]:
import pandas as pd  
import numpy as np
import pandas_profiling
from itertools import combinations 
from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px

## Importación de datos

In [3]:
#datos con el área de las secciones (estimacion feb-2020 con la cartografía de las manzanas)
data_area = pd.read_csv("Datos_secc_area_19.csv", dtype={'Area_Km2':float}) 

#series de tiempo de LNE y PE por seccion (enero 2019 - diciembre 2019)
data_lne = pd.read_csv("Datos_secc2019.csv", parse_dates=['DATE'], dtype={'PE':float, 'LNE':float})

#datos con el tipo de sección (clasificación de la Direccion de Cartografia)
data_tipo_secc = pd.read_csv('Datos_tipo_seccion.txt', sep="|")

#nomenclatura de los estados
data_nom_edo = pd.read_csv('Datos_Nomenclatura_EDO.csv', encoding='utf-8')

#datos con los nombres de los municipios
data_nom_mun = pd.read_csv('Datos_Nomenclatura_MUN.csv', encoding='utf-8')

In [4]:
print(data_tipo_secc.dtypes)
data_tipo_secc

EDO              int64
DTO              int64
MUN              int64
SECC             int64
TIPO_SECCION    object
dtype: object


,EDO,DTO,MUN,SECC,TIPO_SECCION
0,1,1,2,338,M
1,1,1,2,339,M
2,1,1,2,340,R
3,1,1,2,341,M
4,1,1,2,342,R
...,...,...,...,...,...
68431,32,4,57,530,M
68432,32,4,57,531,U
68433,32,4,57,532,U
68434,32,4,57,533,U


In [5]:
print(data_area.dtypes)
data_area

EDO           int64
DTO           int64
MUN           int64
SECC          int64
Area_Km2    float64
dtype: object


,EDO,DTO,MUN,SECC,Area_Km2
0,1,1,2,338,0.506492
1,1,1,2,339,0.692053
2,1,1,2,340,0.990536
3,1,1,2,341,0.507441
4,1,1,2,342,1.437017
...,...,...,...,...,...
61035,32,4,57,529,0.545010
61036,32,4,57,530,0.757383
61037,32,4,57,531,0.548031
61038,32,4,57,532,0.149622


In [6]:
print(data_lne.dtypes)
data_lne

DATE    datetime64[ns]
EDO              int64
DTO              int64
MUN              int64
SECC             int64
PE             float64
LNE            float64
dtype: object


,DATE,EDO,DTO,MUN,SECC,PE,LNE
0,2019-01-01,1,3,1,1,2163.0,2128.0
1,2019-01-01,1,3,1,2,903.0,891.0
2,2019-01-01,1,3,1,3,1710.0,1689.0
3,2019-01-01,1,3,1,4,1413.0,1397.0
4,2019-01-01,1,3,1,5,788.0,782.0
...,...,...,...,...,...,...,...
821125,2019-12-01,32,4,57,530,1671.0,1649.0
821126,2019-12-01,32,4,57,531,1601.0,1581.0
821127,2019-12-01,32,4,57,532,1081.0,1065.0
821128,2019-12-01,32,4,57,533,1183.0,1164.0


In [7]:
print(data_nom_edo.dtypes)
print(data_nom_mun.dtypes)

EDO               int64
NOMBRE_ESTADO    object
dtype: object
EDO                  int64
MUN                  int64
NOMBRE_MUNICIPIO    object
dtype: object


## Limpieza de datos

In [8]:
#ver si hay valores nulos en las series de tiempo data_lne
data_lne.isnull().sum()

DATE    0
EDO     0
DTO     0
MUN     0
SECC    0
PE      0
LNE     0
dtype: int64

In [9]:
#ver si hay valores nulos
data_area.isnull().sum()

EDO         0
DTO         0
MUN         0
SECC        0
Area_Km2    0
dtype: int64

In [10]:
#ver si hay valores nulos
data_tipo_secc.isnull().sum()

EDO             0
DTO             0
MUN             0
SECC            0
TIPO_SECCION    0
dtype: int64

In [11]:
#se eliminan las filas con valores vacíos
#data_lne.dropna()

In [12]:
#Se agrupan los datos por municipios
#se agrupa con las columnas Date, EDO, MUN
datos_lne = data_lne.groupby(['DATE','EDO','MUN']).agg(
    {
        # find sum of PE
        'PE': "sum",
        # find sum of LNE
        'LNE': "sum"
    }
)

#se resetea el index
datos_lne = datos_lne.reset_index()
datos_lne = datos_lne.groupby(['EDO','MUN'])

#se imprimen los primeros diez valores 
datos_lne.head(10)

,DATE,EDO,MUN,PE,LNE
0,2019-01-01,1,1,655233.0,648063.0
1,2019-01-01,1,2,34681.0,34158.0
2,2019-01-01,1,3,47230.0,46711.0
3,2019-01-01,1,4,11763.0,11652.0
4,2019-01-01,1,5,82167.0,81340.0
...,...,...,...,...,...
24885,2019-10-01,32,54,13257.0,13080.0
24886,2019-10-01,32,55,25416.0,25200.0
24887,2019-10-01,32,56,109832.0,108922.0
24888,2019-10-01,32,57,13345.0,13237.0


In [13]:
#datos de enero de todas los municipios
enero=datos_lne.first() #first para tomar los primeros valores correspondientes a enero
enero2019=enero.drop(['DATE'], axis=1) #eliminamos DATE
enero_2019 = enero2019.reset_index() #recuperamos el índice
enero_2019

,EDO,MUN,PE,LNE
0,1,1,655233.0,648063.0
1,1,2,34681.0,34158.0
2,1,3,47230.0,46711.0
3,1,4,11763.0,11652.0
4,1,5,82167.0,81340.0
...,...,...,...,...
2484,32,54,12562.0,12289.0
2485,32,55,23752.0,23218.0
2486,32,56,103724.0,101234.0
2487,32,57,12784.0,12619.0


In [14]:
#datos de diciembre de todas los municipios
diciembre=datos_lne.last()
diciembre2019=diciembre.drop(['DATE'], axis=1)
diciembre_2019 = diciembre2019.reset_index() #recuperamos el índice
diciembre_2019

,EDO,MUN,PE,LNE
0,1,1,658689.0,648368.0
1,1,2,34722.0,34222.0
2,1,3,46896.0,46424.0
3,1,4,11789.0,11623.0
4,1,5,83556.0,82404.0
...,...,...,...,...
2484,32,54,13455.0,13004.0
2485,32,55,25529.0,25182.0
2486,32,56,110286.0,108728.0
2487,32,57,13385.0,13190.0


**Procesamiento datos área urbanizada**

In [15]:
#Se agrupan los datos por municipios
#se agrupa con las columnas Date, EDO, MUN
data_area = data_area.groupby(['EDO','MUN']).agg(
    {
        # find sum of PE
        'Area_Km2': "sum"
    }
)

#se resetea el index
data_area = data_area.reset_index()
data_area

,EDO,MUN,Area_Km2
0,1,1,113.416597
1,1,2,23.049424
2,1,3,19.881414
3,1,4,4.311295
4,1,5,27.645669
...,...,...,...
2451,32,54,9.152572
2452,32,55,24.268153
2453,32,56,35.459433
2454,32,57,6.569019


**Procesamiento datos tipo de sección**

In [16]:
data_tipo_secc=data_tipo_secc.groupby(['EDO','MUN'])['TIPO_SECCION'].apply(lambda x: (x=='R').sum()).reset_index(name='NUM_SECC_RUR')
data_tipo_secc

,EDO,MUN,NUM_SECC_RUR
0,1,1,9
1,1,2,11
2,1,3,14
3,1,4,2
4,1,5,6
...,...,...,...
2453,32,54,21
2454,32,55,33
2455,32,56,11
2456,32,57,4


## Creación de los indicadores base (_feature engineering_)

A partir de las bases de datos, se procesan y construyen los ocho indicadores base:

In [17]:
#calculamos los coeficientes de variacion del PE y la LNE para todas los municipios
datos_vc = datos_lne.agg({'PE': lambda x: np.std(x)/np.mean(x),'LNE': lambda x: np.std(x)/np.mean(x)}).rename(columns={'PE': 'Coef_Var_PE', 'LNE': 'Coef_Var_LNE'})
datos_vc

Coef_Var_PE  Coef_Var_LNE
EDO MUN                           
1   1       0.008283      0.011367
    2       0.008083      0.013432
    3       0.009902      0.013266
    4       0.006601      0.011840
    5       0.009471      0.010581
...              ...           ...
32  54      0.021009      0.021333
    55      0.022179      0.025415
    56      0.018674      0.021654
    57      0.015167      0.015656
    58      0.015562      0.015360

[2489 rows x 2 columns]

In [18]:
#calculamos los coeficientes de autocorrelacion (lag=1month) para todos los municipios
datos_autcorr = datos_lne.agg({'PE': lambda x: x.autocorr(lag=1),'LNE': lambda x: x.autocorr(lag=1)}).rename(columns={'PE': 'Autocorr_PE', 'LNE': 'Autocorr_LNE'})
datos_autcorr

Autocorr_PE  Autocorr_LNE
EDO MUN                           
1   1       0.601130      0.419435
    2       0.581193      0.374738
    3       0.555058      0.484059
    4       0.596679      0.390646
    5       0.789590      0.422715
...              ...           ...
32  54      0.934105      0.861310
    55      0.997445      0.992794
    56      0.998872      0.992407
    57      0.989206      0.962100
    58      0.904901      0.954892

[2489 rows x 2 columns]

In [19]:
#calculamos la razón media de LNE/PE para todos los municipios
data_lne['Razon_LNE_PE'] = data_lne['LNE']/data_lne['PE'] #calculamos la razón para todos los municipios y meses
datos_ratio=data_lne.groupby(['EDO','MUN'])['Razon_LNE_PE'].mean() 
datos_ratio

EDO  MUN
1    1      0.990718
     2      0.989080
     3      0.991150
     4      0.989896
     5      0.991134
              ...   
32   54     0.977370
     55     0.987283
     56     0.988428
     57     0.986531
     58     0.989614
Name: Razon_LNE_PE, Length: 2489, dtype: float64

In [20]:
#calculamos la tasa de crecimiento de la LNE entre enero2019 y diciembre2019
diciembre_2019['TC_LNE_2019'] = (diciembre_2019['LNE'] - enero_2019['LNE'])/enero_2019['LNE']
diciembre_2019

,EDO,MUN,PE,LNE,TC_LNE_2019
0,1,1,658689.0,648368.0,0.000471
1,1,2,34722.0,34222.0,0.001874
2,1,3,46896.0,46424.0,-0.006144
3,1,4,11789.0,11623.0,-0.002489
4,1,5,83556.0,82404.0,0.013081
...,...,...,...,...,...
2484,32,54,13455.0,13004.0,0.058182
2485,32,55,25529.0,25182.0,0.084590
2486,32,56,110286.0,108728.0,0.074027
2487,32,57,13385.0,13190.0,0.045249


## Función para calcular _Proportional Variability_

In [21]:
#Función para calcular PV (Variacion Proporcional)
def PVar(lista):
    n = len(lista)
    CC= 2/(n*(n-1))
    sumaD = sum([(abs(i-j)/max(i,j)) for i,j in list(combinations(lista, 2))])
    return CC*sumaD

In [22]:
#ejemplo
array = np.arange(20).reshape(4,5)
print(array)
lista= array[:,1]
lista

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


array([ 1,  6, 11, 16])

In [23]:
PVar(lista)

0.6786616161616161

In [24]:
#ejemplo
a = pd.DataFrame(array)
print(a)
print(list(a.columns)) 

    0   1   2   3   4
0   0   1   2   3   4
1   5   6   7   8   9
2  10  11  12  13  14
3  15  16  17  18  19
[0, 1, 2, 3, 4]


In [25]:
a[1]

0     1
1     6
2    11
3    16
Name: 1, dtype: int32

In [26]:
a.agg({1: lambda x: PVar(x)})

1    0.678662
dtype: float64

In [27]:
#calculamos la proportional variability para cada municipio
datos_PV = datos_lne.agg({'PE': lambda x: PVar(x),'LNE': lambda x: PVar(x)}).rename(columns={'PE': 'Var_Prop_PE', 'LNE': 'Var_Prop_LNE'})
datos_PV

Var_Prop_PE  Var_Prop_LNE
EDO MUN                           
1   1       0.009498      0.012420
    2       0.009252      0.015397
    3       0.011726      0.015719
    4       0.007624      0.013461
    5       0.011288      0.011357
...              ...           ...
32  54      0.024748      0.024711
    55      0.026791      0.029953
    56      0.022578      0.025153
    57      0.018296      0.019180
    58      0.018134      0.017680

[2489 rows x 2 columns]

## Unión de los datasets

In [28]:
#se unen coeficiente de variación y proportional variability
datos1 = datos_vc.join(datos_PV, lsuffix='_caller', rsuffix='_other')
datos1

Coef_Var_PE  Coef_Var_LNE  Var_Prop_PE  Var_Prop_LNE
EDO MUN                                                      
1   1       0.008283      0.011367     0.009498      0.012420
    2       0.008083      0.013432     0.009252      0.015397
    3       0.009902      0.013266     0.011726      0.015719
    4       0.006601      0.011840     0.007624      0.013461
    5       0.009471      0.010581     0.011288      0.011357
...              ...           ...          ...           ...
32  54      0.021009      0.021333     0.024748      0.024711
    55      0.022179      0.025415     0.026791      0.029953
    56      0.018674      0.021654     0.022578      0.025153
    57      0.015167      0.015656     0.018296      0.019180
    58      0.015562      0.015360     0.018134      0.017680

[2489 rows x 4 columns]

In [29]:
#se agrega autocorrelacion
datos2019_mun = datos1.join(datos_autcorr, lsuffix='_caller', rsuffix='_other')
datos2019_mun

Coef_Var_PE  Coef_Var_LNE  Var_Prop_PE  Var_Prop_LNE  Autocorr_PE  \
EDO MUN                                                                      
1   1       0.008283      0.011367     0.009498      0.012420     0.601130   
    2       0.008083      0.013432     0.009252      0.015397     0.581193   
    3       0.009902      0.013266     0.011726      0.015719     0.555058   
    4       0.006601      0.011840     0.007624      0.013461     0.596679   
    5       0.009471      0.010581     0.011288      0.011357     0.789590   
...              ...           ...          ...           ...          ...   
32  54      0.021009      0.021333     0.024748      0.024711     0.934105   
    55      0.022179      0.025415     0.026791      0.029953     0.997445   
    56      0.018674      0.021654     0.022578      0.025153     0.998872   
    57      0.015167      0.015656     0.018296      0.019180     0.989206   
    58      0.015562      0.015360     0.018134      0.017680     0.904901   

         Autocorr_LNE  
EDO MUN                
1   1        0.419435  
    2        0.374738  
    3        0.484059  
    4        0.390646  
    5        0.422715  
...               ...  
32  54       0.861310  
    55       0.992794  
    56       0.992407  
    57       0.962100  
    58       0.954892  

[2489 rows x 6 columns]

In [30]:
#se agrega razon media LNE/PE
datos2019_mun = datos2019_mun.join(datos_ratio, lsuffix='_caller', rsuffix='_other')
datos2019_mun

Coef_Var_PE  Coef_Var_LNE  Var_Prop_PE  Var_Prop_LNE  Autocorr_PE  \
EDO MUN                                                                      
1   1       0.008283      0.011367     0.009498      0.012420     0.601130   
    2       0.008083      0.013432     0.009252      0.015397     0.581193   
    3       0.009902      0.013266     0.011726      0.015719     0.555058   
    4       0.006601      0.011840     0.007624      0.013461     0.596679   
    5       0.009471      0.010581     0.011288      0.011357     0.789590   
...              ...           ...          ...           ...          ...   
32  54      0.021009      0.021333     0.024748      0.024711     0.934105   
    55      0.022179      0.025415     0.026791      0.029953     0.997445   
    56      0.018674      0.021654     0.022578      0.025153     0.998872   
    57      0.015167      0.015656     0.018296      0.019180     0.989206   
    58      0.015562      0.015360     0.018134      0.017680     0.904901   

         Autocorr_LNE  Razon_LNE_PE  
EDO MUN                              
1   1        0.419435      0.990718  
    2        0.374738      0.989080  
    3        0.484059      0.991150  
    4        0.390646      0.989896  
    5        0.422715      0.991134  
...               ...           ...  
32  54       0.861310      0.977370  
    55       0.992794      0.987283  
    56       0.992407      0.988428  
    57       0.962100      0.986531  
    58       0.954892      0.989614  

[2489 rows x 7 columns]

In [31]:
#se recupera el índice (desagrupa)
datos2019_mun= datos2019_mun.reset_index()
datos2019_mun

,EDO,MUN,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE
0,1,1,0.008283,0.011367,0.009498,0.012420,0.601130,0.419435,0.990718
1,1,2,0.008083,0.013432,0.009252,0.015397,0.581193,0.374738,0.989080
2,1,3,0.009902,0.013266,0.011726,0.015719,0.555058,0.484059,0.991150
3,1,4,0.006601,0.011840,0.007624,0.013461,0.596679,0.390646,0.989896
4,1,5,0.009471,0.010581,0.011288,0.011357,0.789590,0.422715,0.991134
...,...,...,...,...,...,...,...,...,...
2484,32,54,0.021009,0.021333,0.024748,0.024711,0.934105,0.861310,0.977370
2485,32,55,0.022179,0.025415,0.026791,0.029953,0.997445,0.992794,0.987283
2486,32,56,0.018674,0.021654,0.022578,0.025153,0.998872,0.992407,0.988428
2487,32,57,0.015167,0.015656,0.018296,0.019180,0.989206,0.962100,0.986531


In [32]:
#datos al corte de diciembre 2019
diciembre_2019

,EDO,MUN,PE,LNE,TC_LNE_2019
0,1,1,658689.0,648368.0,0.000471
1,1,2,34722.0,34222.0,0.001874
2,1,3,46896.0,46424.0,-0.006144
3,1,4,11789.0,11623.0,-0.002489
4,1,5,83556.0,82404.0,0.013081
...,...,...,...,...,...
2484,32,54,13455.0,13004.0,0.058182
2485,32,55,25529.0,25182.0,0.084590
2486,32,56,110286.0,108728.0,0.074027
2487,32,57,13385.0,13190.0,0.045249


In [33]:
#se agregan los datos de PE, LNE (diciembre2019) y la tasa de crecimiento anual 2019 de la LNE
datos2019_mun_vf = pd.merge(datos2019_mun, diciembre_2019, on=['EDO','MUN'], how='inner')
datos2019_mun_vf

,EDO,MUN,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019
0,1,1,0.008283,0.011367,0.009498,0.012420,0.601130,0.419435,0.990718,658689.0,648368.0,0.000471
1,1,2,0.008083,0.013432,0.009252,0.015397,0.581193,0.374738,0.989080,34722.0,34222.0,0.001874
2,1,3,0.009902,0.013266,0.011726,0.015719,0.555058,0.484059,0.991150,46896.0,46424.0,-0.006144
3,1,4,0.006601,0.011840,0.007624,0.013461,0.596679,0.390646,0.989896,11789.0,11623.0,-0.002489
4,1,5,0.009471,0.010581,0.011288,0.011357,0.789590,0.422715,0.991134,83556.0,82404.0,0.013081
...,...,...,...,...,...,...,...,...,...,...,...,...
2484,32,54,0.021009,0.021333,0.024748,0.024711,0.934105,0.861310,0.977370,13455.0,13004.0,0.058182
2485,32,55,0.022179,0.025415,0.026791,0.029953,0.997445,0.992794,0.987283,25529.0,25182.0,0.084590
2486,32,56,0.018674,0.021654,0.022578,0.025153,0.998872,0.992407,0.988428,110286.0,108728.0,0.074027
2487,32,57,0.015167,0.015656,0.018296,0.019180,0.989206,0.962100,0.986531,13385.0,13190.0,0.045249


In [34]:
#se unen los datos de area y tipo de sección
data_area_tipo = pd.merge(data_tipo_secc, data_area, on=['EDO','MUN'], how='inner')
data_area_tipo

,EDO,MUN,NUM_SECC_RUR,Area_Km2
0,1,1,9,113.416597
1,1,2,11,23.049424
2,1,3,14,19.881414
3,1,4,2,4.311295
4,1,5,6,27.645669
...,...,...,...,...
2451,32,54,21,9.152572
2452,32,55,33,24.268153
2453,32,56,11,35.459433
2454,32,57,4,6.569019


In [35]:
#se unen los datos de area_tipo sección con los datos de feature engineering
datos2019_mun_vf2 = pd.merge(datos2019_mun_vf, data_area_tipo, on=['EDO','MUN'], how='inner')
datos2019_mun_vf2

,EDO,MUN,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019,NUM_SECC_RUR,Area_Km2
0,1,1,0.008283,0.011367,0.009498,0.012420,0.601130,0.419435,0.990718,658689.0,648368.0,0.000471,9,113.416597
1,1,2,0.008083,0.013432,0.009252,0.015397,0.581193,0.374738,0.989080,34722.0,34222.0,0.001874,11,23.049424
2,1,3,0.009902,0.013266,0.011726,0.015719,0.555058,0.484059,0.991150,46896.0,46424.0,-0.006144,14,19.881414
3,1,4,0.006601,0.011840,0.007624,0.013461,0.596679,0.390646,0.989896,11789.0,11623.0,-0.002489,2,4.311295
4,1,5,0.009471,0.010581,0.011288,0.011357,0.789590,0.422715,0.991134,83556.0,82404.0,0.013081,6,27.645669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,32,54,0.021009,0.021333,0.024748,0.024711,0.934105,0.861310,0.977370,13455.0,13004.0,0.058182,21,9.152572
2452,32,55,0.022179,0.025415,0.026791,0.029953,0.997445,0.992794,0.987283,25529.0,25182.0,0.084590,33,24.268153
2453,32,56,0.018674,0.021654,0.022578,0.025153,0.998872,0.992407,0.988428,110286.0,108728.0,0.074027,11,35.459433
2454,32,57,0.015167,0.015656,0.018296,0.019180,0.989206,0.962100,0.986531,13385.0,13190.0,0.045249,4,6.569019


In [36]:
#calculamos la densidad por municipio (al corte de diciembre 2019)
datos2019_mun_vf2['Densidad_LNE'] = datos2019_mun_vf2['LNE']/datos2019_mun_vf2['Area_Km2'] 
datos2019_mun_vf2

,EDO,MUN,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019,NUM_SECC_RUR,Area_Km2,Densidad_LNE
0,1,1,0.008283,0.011367,0.009498,0.012420,0.601130,0.419435,0.990718,658689.0,648368.0,0.000471,9,113.416597,5716.694188
1,1,2,0.008083,0.013432,0.009252,0.015397,0.581193,0.374738,0.989080,34722.0,34222.0,0.001874,11,23.049424,1484.722551
2,1,3,0.009902,0.013266,0.011726,0.015719,0.555058,0.484059,0.991150,46896.0,46424.0,-0.006144,14,19.881414,2335.045131
3,1,4,0.006601,0.011840,0.007624,0.013461,0.596679,0.390646,0.989896,11789.0,11623.0,-0.002489,2,4.311295,2695.941455
4,1,5,0.009471,0.010581,0.011288,0.011357,0.789590,0.422715,0.991134,83556.0,82404.0,0.013081,6,27.645669,2980.720064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,32,54,0.021009,0.021333,0.024748,0.024711,0.934105,0.861310,0.977370,13455.0,13004.0,0.058182,21,9.152572,1420.802802
2452,32,55,0.022179,0.025415,0.026791,0.029953,0.997445,0.992794,0.987283,25529.0,25182.0,0.084590,33,24.268153,1037.656238
2453,32,56,0.018674,0.021654,0.022578,0.025153,0.998872,0.992407,0.988428,110286.0,108728.0,0.074027,11,35.459433,3066.264498
2454,32,57,0.015167,0.015656,0.018296,0.019180,0.989206,0.962100,0.986531,13385.0,13190.0,0.045249,4,6.569019,2007.910029


## Metadatos y datos procesados

In [37]:
#se combinan las nomenclaturas de estados y municipios
nomenclatura=pd.merge(data_nom_edo,data_nom_mun, on=['EDO'], how='inner')
nomenclatura

,EDO,NOMBRE_ESTADO,MUN,NOMBRE_MUNICIPIO
0,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO
1,1,AGUASCALIENTES,8,SAN JOSE DE GRACIA
2,1,AGUASCALIENTES,9,TEPEZALA
3,1,AGUASCALIENTES,11,EL LLANO
4,1,AGUASCALIENTES,2,ASIENTOS
...,...,...,...,...
2453,32,ZACATECAS,45,TEPECHITLAN
2454,32,ZACATECAS,14,GENERAL FRANCISCO R. MURGUIA
2455,32,ZACATECAS,44,TABASCO
2456,32,ZACATECAS,9,CHALCHIHUITES


In [38]:
#se concatenan los nombre de los estados y municipios con los resultados:
datos_procesados=pd.merge(nomenclatura, datos2019_mun_vf2, on=['EDO','MUN'], how='inner')
datos_procesados

,EDO,NOMBRE_ESTADO,MUN,NOMBRE_MUNICIPIO,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019,NUM_SECC_RUR,Area_Km2,Densidad_LNE
0,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO,0.017131,0.015085,0.020041,0.017733,0.965775,0.755939,0.989272,35518.0,34917.0,0.037498,3,9.791576,3566.024547
1,1,AGUASCALIENTES,8,SAN JOSE DE GRACIA,0.005040,0.008345,0.005584,0.008887,0.435676,0.237061,0.991803,7004.0,6914.0,0.000289,4,3.241104,2133.223463
2,1,AGUASCALIENTES,9,TEPEZALA,0.006879,0.011777,0.007903,0.011921,0.605737,0.344567,0.990635,15733.0,15545.0,0.005108,6,6.271495,2478.675412
3,1,AGUASCALIENTES,11,EL LLANO,0.008917,0.012239,0.009983,0.014232,0.585621,0.504020,0.991016,15298.0,15067.0,-0.003373,6,10.006312,1505.749552
4,1,AGUASCALIENTES,2,ASIENTOS,0.008083,0.013432,0.009252,0.015397,0.581193,0.374738,0.989080,34722.0,34222.0,0.001874,11,23.049424,1484.722551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,32,ZACATECAS,45,TEPECHITLAN,0.014994,0.019408,0.017748,0.021905,0.987834,0.959378,0.983523,7061.0,6959.0,0.066677,13,13.507234,515.205401
2452,32,ZACATECAS,14,GENERAL FRANCISCO R. MURGUIA,0.019415,0.022769,0.022931,0.026072,0.964977,0.984897,0.986450,17610.0,17372.0,0.082705,32,17.129462,1014.159112
2453,32,ZACATECAS,44,TABASCO,0.027363,0.029823,0.032716,0.035416,0.992677,0.993335,0.983221,12689.0,12527.0,0.100307,16,8.620183,1453.217492
2454,32,ZACATECAS,9,CHALCHIHUITES,0.016451,0.021097,0.019649,0.024771,0.975279,0.988608,0.982861,8339.0,8222.0,0.075052,10,13.707054,599.837116


In [39]:
#se guarda el último dataframe (datos procesados) en un csv
#datos_procesados.to_csv(r'Datos-procesados_mun2019.csv', index = None)

<br>

## Referencias
- https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/
- https://plot.ly/python/v3/ipython-notebooks/principal-component-analysis/#pca-vs-lda
- https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python
- https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
- https://alexanderfabisch.github.io/t-sne-in-scikit-learn.html
- https://medium.com/@indreshbhattacharyya/feature-selection-categorical-feature-selection-boruta-light-gbm-chi-square-bf47e94e2558
- https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
- https://stackoverflow.com/questions/52472757/creating-a-boxplot-facetgrid-in-seaborn-for-python
- https://www.kdnuggets.com/2019/07/10-simple-hacks-speed-data-analysis-python.html
- https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet